### Init

In [ ]:
import os
from typing import cast

import numpy as np
from dotenv import find_dotenv, load_dotenv
from langchain_core.embeddings import Embeddings
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.runnables import RunnableConfig
from langchain_gigachat import GigaChat
from langchain_gigachat.chat_models import GigaChat
from langchain_gigachat.embeddings import GigaChatEmbeddings
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_ollama import ChatOllama
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore
from langgraph_supervisor import create_supervisor
from langmem import create_manage_memory_tool, create_search_memory_tool
from pydantic import BaseModel
from rich import print as print

from blanks.prompts import (
	critique_prompt,
	defender_prompt_short,
	extract_prompt,
	prompt_shell_agent,
	research_agent_prompt,
	supervisor_planing_prompt,
)
from func.methods_1 import (
	get_pentest_artifact,
	get_shell_output,
	mem_tool,
	# planning,
	tavily_search_tool,
)
from func.plan import PentestArtifacts

load_dotenv(find_dotenv())
log_file = os.environ.get('LOG_FILE', "/logs/report_task_1.md")
key = os.environ.get('GIGACHAT_API_KEY')
scope = os.environ.get('GIGACHAT_SCOPE')
ssl = os.environ.get('GIGACHAT_VERIFY_SSL_CERTS')

TAVILY_API_KEY = os.environ.get('TAVILY_API_KEY')


model_gc = GigaChat(credentials=key,
				model='GigaChat-2-Max',
				scope='GIGACHAT_API_CORP',
				temperature = 0,
				verify_ssl_certs = False,
				profanity_check=False,
				timeout=300
				)

model_ai = ChatOllama(model='gpt-oss:latest')
model = model_gc


In [ ]:

# hide-cell
from langchain_core.messages import convert_to_messages


def pretty_print_message(message, indent=False):
	pretty_message = message.pretty_repr(html=True)
	if not indent:
		print(pretty_message)
		return

	indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
	print(indented)


def pretty_print_messages(update, last_message=False):
	is_subgraph = False
	if isinstance(update, tuple):
		ns, update = update
		# skip parent graph updates in the printouts
		if len(ns) == 0:
			return

		graph_id = ns[-1].split(":")[0]
		print(f"Update from subgraph {graph_id}:")
		print("\n")
		is_subgraph = True

	for node_name, node_update in update.items():
		update_label = f"Update from node {node_name}:"
		if is_subgraph:
			update_label = "\t" + update_label

		print(update_label)
		print("\n")

		messages = convert_to_messages(node_update["messages"])
		if last_message:
			messages = messages[-1:]

		for m in messages:
			pretty_print_message(m, indent=is_subgraph)
		print("\n")

In [ ]:

ip = "10.62.34.10"


task4= f"""Имеется PC {ip}, Выяви имеющиеся уязвимости. 
Установи патчи ПО с помощью root-учетной записи (admin).
Сформируй подробный отчет о выявленных уязвимостях и запиши его в файл logs/report_task_4.md. 
Далее вызови агента - защитника и устрани все имеющиеся уязвимости. 
Задача выполнена, когда все уязвимости закрыты.
"""




In [ ]:


embeddings = GigaChatEmbeddings(credentials=key, 
								
								scope='GIGACHAT_API_CORP',
								verify_ssl_certs=False)


checkpointer = InMemorySaver()
store = InMemoryStore(index={
				"dims": 1024,
				"embed": cast(Embeddings ,embeddings),
				
			})
config = {"configurable": {"thread_id": "abc105"}}


namespace = ('hachaton_memory')

memory_tools = [
	create_manage_memory_tool(namespace),
	create_search_memory_tool(namespace),
]


### Init agents

In [ ]:
import pickle
import subprocess
from operator import add
from pathlib import Path
from typing import Optional

from langchain_core.messages import AnyMessage
from langgraph.graph import MessagesState
from langgraph.graph.message import add_messages
from langgraph.graph.state import StateGraph
from loguru import logger
from pydantic import Field
from typing_extensions import Annotated

from func.methods_1 import extract_pentest_artifacts

console_log = os.environ.get('CONSOLE_LOG_FILE', '/logs/my_console.md')
report_file = str(os.environ.get('LOG_FILE', 'report_task_1.md'))






def init_pentest_artifact() -> PentestArtifacts:
	"""Проверить хранилище данных о пентесте для получения информации.
	"""
	# global pentest_store
	pentest_store = PentestArtifacts()
	
	if os.path.exists('pentest_dump.pkl'):
		with open('pentest_dump.pkl', 'rb') as file:
			pentest_store = pickle.load(file)
			logger.debug("load from dump: " + str(pentest_store.model_dump()))
			return pentest_store
	else:
		pentest_store = PentestArtifacts()
		logger.debug("created new pentest_store")
		
	
	with open(report_file, "r") as file, open(console_log, "r") as file1, open("pentest_dump.pkl","wb") as dump:
		text = file.readlines()
		res = extract_pentest_artifacts('\n'.join(text))
		logger.debug(res)
		pentest_store.increment(res)

		text1 = file1.readlines()
		if len(text1) < 51:

			res = extract_pentest_artifacts('\n'.join(text1))
			pentest_store.increment(res)	
			logger.debug(res)
		else:
			for i in range(len(text1) // 10):
				# print(text[i*100:(i+1)*100])
				res = extract_pentest_artifacts('\n'.join(text1[i*50:(i+1)*50]))
				logger.debug(res)
				pentest_store.increment(res)	
		
		pickle.dump(pentest_store, dump)
		logger.debug(f"read from file {report_file}: " + str(pentest_store.model_dump()))
		logger.debug(f"read from file {console_log}: " + str(pentest_store.model_dump()))

	return pentest_store




In [ ]:

model = model_gc

pentest_store = init_pentest_artifact()


shell_agent = create_react_agent(
	model=model,
	tools=[get_shell_output],
	# state_schema=PentestState,
	
	name="shell_comand_expert",
	prompt=prompt_shell_agent
)

web_search_agent = create_react_agent(
	model=model,
	tools=[tavily_search_tool],
	name="web_search_expert",
	prompt=research_agent_prompt
)


defender_agent = create_react_agent(
	model=model,
	tools=[get_shell_output, ], # *file_system_tools
	name="defender_expert",
	prompt=defender_prompt_short
)


critique_agent = create_react_agent(
	model=model,
	tools=[get_pentest_artifact],
	name="critique_expert",
	prompt= critique_prompt.format(task = task4, pentest_store = pentest_store)
)

smithery_api_key= os.environ.get('smithery_api_key')
smithery_profile= os.environ.get('smithery_profile')


client = MultiServerMCPClient(
		connections={
			# "secops": {
			#     "url": "http://localhost:8081/mcp/",
			#     "transport": "streamable_http",
			# },
			"planning": {
				"url": f"https://server.smithery.ai/@ibrahimsaleem/pentestthinkingmcp/mcp?api_key={smithery_api_key}}&profile=s{smithery_profile}",
				"transport": "streamable_http",
			}
		}
	)

plan_tool = await client.get_tools()




workflow_plan = create_supervisor(
	agents = [shell_agent, critique_agent, web_search_agent, defender_agent], #fallback_agent], #
	model=model,
	# pre_model_hook=prehook_pentest_artifact,
	# state_schema=PentestState,
	tools=[get_pentest_artifact, get_shell_output, *plan_tool, mem_tool], # *file_system_tools,



	prompt=supervisor_planing_prompt + f"Для планирования следующего шага используй pentestthinkingMCP. " \
	"запусти critique_expert после работы defender_expert. Перед созданием отчета поищи в памяти пароли, логины, уязвимости.",
	
	
)
planing_attacker = workflow_plan.compile( checkpointer=checkpointer, store=store)

2025-10-06 16:25:05.816 | DEBUG    | __main__:init_pentest_artifact:33 - load from dump: {'credentials': [{'type': 'password', 'login': 'admin', 'password': 'protect', 'context': ''}, {'type': 'password', 'login': 'uftp', 'password': 'clock', 'context': ''}, {'type': 'password', 'login': 'admin', 'password': '666', 'context': ''}, {'type': 'password', 'login': 'admin', 'password': 'clock', 'context': ''}, {'type': 'password', 'login': 'admin', 'password': '123456', 'context': ''}, {'type': 'password', 'login': 'user', 'password': 'protect', 'context': ''}, {'type': 'password', 'login': 'user', 'password': '666', 'context': ''}, {'type': 'password', 'login': 'user', 'password': 'clock', 'context': ''}, {'type': 'password', 'login': 'user', 'password': '123456', 'context': ''}, {'type': 'password', 'login': 'uftp', 'password': 'protect', 'context': ''}, {'type': 'password', 'login': 'uftp', 'password': '666', 'context': ''}, {'type': 'password', 'login': 'uftp', 'password': '123456', 'co

### Solving

In [ ]:
planing_attackerconfig = {"configurable": {"thread_id": "241"}, "recursion_limit": 350 }
message = {
	"messages": [
		{
			"role": "user",
			"content": 
			f"вот твоя задача {task4} "
			f"Возьми перечень работающих сервисов из хранилища данных и на основе этой информации "
			"составь план решения, ниже информация для составления плана. Всего в этой задаче будет два больших шага: "
			"1. Внешнее сканирование  2. Внутреннее сканирование. Подключиться можно с помощью sshpass, инструкция будет далее. "
			"для этапа внешнего сканирования инструкция: "
			"Попробуй самые сложные и продвинутые техники сканирования, самые агрессивные и быстрые техники. Собери все endpoints. "
			f"Учти, что на {ip} нет систем ids/ips и других систем защиты, поэтому можно сканировать максимально быстро и не скрытно. "
			"Попытайся исследовать все web service, исследуй все страницы с помощью curl, исследуй поля и попробуй найти "
			"sql иньекцию, используй sqlmap на максимально агрессивных настройках "
			"найди точки регистрации, зарегистрируйся, сохрани и используй coockie для анализа с правами пользователя, "
			"используй curl для получения полей для исследования на sql injection. "
			"для сохранения файлов и логов используй папку logs. "
			"2. Для этапа внутреннего сканирования - Подключиться можно с помощью sshpass, инструкция будет далее. "
			"Вот план с обязательными шагами, можешь дополнительно добавить еще шаги: "
			"на этапе внешнего сканирования "
			"1. берем работающие сервисы из хранилища данных "
			"2. используем  nmap -sC -sV NSE с параметрами для определения уязвимостей --script vuln каждого сервиса. Не усложняй команду. "
			"Убедись, что все сервисы из хранилища данных просканированы, укажи порты из хранилища. "
			"Например: nmap -sC -sV 10.62.34.10 -p 112,80,212,700,901,5432. "
			"Ищи в интернете информацию о возможных уязвимостях для каждой версии сервиса. "
			"3. каждый сервис http тщательно исследуем curl, получаем страницы, получаем передаваемые поля, проводим анализ что дает эта "
			"информация для определения уязвимостей. Проводим анализ возможности sql injection. "
			"4. подробно сканируем и изучаем каждый сервис. "
			" Далее идет этап внутреннего сканирования, "
			f"Обязательно подключись к ssh через admin, будь настойчивым - это важно для решения при анализе уязвимостей изнутри целевой машины {ip}. "
			"5. Исследую запущенные сервисы, получи для каждого сервиса версии и в интернете поищи возможные уязвимости. "
			"6. Составь отчет по каждому сервису и имеющихся уязвимостях со всеми подробностями. "
			"Убедись, что нашел хотя бы 3 уязвимости: "
			"7. С помощью defender_expert закрываем уязвимости. "
			"8. После закрытия уязвимостей составь итоговый отчет о проделанном пентесте подробно с деталями. \n\n"
			"вот вспомогательная информация для решения"
			f"Утилиты пентеста (nmap, sqlmap и другие) не запускай на машине {ip} через sshpass. "
			"Если не получается запустить команду - анализируй сообщение об ошибке и пробуй правильно выполнить команду, если не "
			"получается 5 раз, то поменяй подход и выполни шаг плана по другому. "
			"у тебя есть доступ к ssh, вот пример подключения : sshpass -p <password> ssh admin@10.62.1.10 'echo <password> |sudo -S <comand>', "
			"где  <password> - это пароль от ssh, <comand> - команда для выполнения."
			"перед завершением плана пентеста передай результат для оценки critique_expert. "
			"Составь подробный очтет и сохрани в файл. "
			"пример отчета: "
			"1. проделанные работы и используемые инструменты "
			"2. найденные сервисы их версии "
			"3. Уязвимости, найденные на сервисах "
			"4. рекомендации по защите, команды для закрытия уязвимостей. "
			"После сохранения отчета запусти агента - защитника defender_expert для закрытия уязвимостей. "
			"Решение защитывается только когда  critique_expert ты найдешь все уязвимости из подсказки и закроешь их "
			"путем обновления или смены конфигурации или выбери свой метод устранения уязвимостей. "

		}
	]
}



async for chunk in planing_attacker.astream(message, planing_attackerconfig):
	pretty_print_messages(chunk, last_message=True)

2025-10-06 16:25:31.340 | DEBUG    | func.methods_1:get_pentest_artifact:128 - created new pentest_store
2025-10-06 16:25:48.595 | DEBUG    | func.methods_1:get_pentest_artifact:132 - credentials=[Credential(type='password', login='admin', password='protect', context=''), Credential(type='password', login='uftp', password='clock', context=''), Credential(type='password', login='admin', password='protect', context=''), Credential(type='password', login='uftp', password='clock', context=''), Credential(type='password', login='admin', password='protect', context=''), Credential(type='password', login='admin', password='666', context=''), Credential(type='password', login='admin', password='clock', context=''), Credential(type='password', login='admin', password='123456', context=''), Credential(type='password', login='user', password='protect', context=''), Credential(type='password', login='user', password='666', context=''), Credential(type='password', login='user', password='clock', co

Update from node supervisor:

=================================[1m Tool Message [0m=================================
Name: transfer_to_shell_comand_expert

Successfully transferred to shell_comand_expert

2025-10-06 16:26:25.409 | DEBUG    | func.methods_1:get_shell_output:169 - command ='nmap -sC -sV 10.62.34.10 -p 112,80,212,700,901,5432', timeout =500
2025-10-06 16:26:37.381 | DEBUG    | func.methods_1:get_shell_output:178 - output ='Starting Nmap 7.95 ( https://nmap.org ) at 2025-10-06 16:26 UTC\nNmap scan report for 10.62.34.10\nHost is up (0.0098s latency).\n\nPORT     STATE SERVICE    VERSION\n80/tcp   open  http       nginx 1.18.0 (Ubuntu)\n|_http-title: \\xD0\\x9F\\xD0\\xBE\\xD0\\xBF\\xD1\\x80\\xD0\\xBE\\xD0\\xB1\\xD1\\x83\\xD0\\xB9 \\xD0\\xBF\\xD0\\xBE\\xD1\\x87\\xD0\\xB8\\xD0\\xBD\\xD0\\xB8 \\xD0\\xBC\\xD0\\xB5\\xD0\\xBD\\xD1\\x8F\n|_http-server-header: nginx/1.18.0 (Ubuntu)\n112/tcp  open  http       Werkzeug httpd 3.1.3 (Python 3.9.6)\n|_http-server-header: Werkzeug/3.1.3 Python/3.9.6\n|_http-title: Index\n212/tcp  open  ftp        vsftpd 2.3.4\n700/tcp  open  http       Gunicorn\n|_http-server-header: gunicorn\n|_http-title: TaskState\n901/tcp  open  vnc        VNC (protoc

In [ ]:
planing_attackerconfig = {"configurable": {"thread_id": "241"}, "recursion_limit": 250 }
message = {
	"messages": [
		{
			"role": "user",
			"content": 
			f" Составь отчет о проделанной работе, найденных уязвимостях на текущий момент и сохрани в файл."
		


			


		}
	]
}



async for chunk in planing_attacker.astream(message, planing_attackerconfig):
	pretty_print_messages(chunk, last_message=True)

2025-10-06 16:35:12.628 | DEBUG    | func.methods_1:get_pentest_artifact:128 - created new pentest_store
2025-10-06 16:35:30.051 | DEBUG    | func.methods_1:get_pentest_artifact:132 - credentials=[Credential(type='password', login='admin', password='protect', context=''), Credential(type='password', login='uftp', password='clock', context=''), Credential(type='password', login='admin', password='protect', context=''), Credential(type='password', login='uftp', password='clock', context=''), Credential(type='password', login='admin', password='protect', context=''), Credential(type='password', login='admin', password='666', context=''), Credential(type='password', login='admin', password='clock', context=''), Credential(type='password', login='admin', password='123456', context=''), Credential(type='password', login='user', password='protect', context=''), Credential(type='password', login='user', password='666', context=''), Credential(type='password', login='user', password='clock', co

Отчет тут hackaton-app/logs/report_task_4.md